In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import os
import pandas as pd
from transformers import AutoConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from google.colab import drive
import numpy as np

In [ ]:
!pip install transformers

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# 새로운 리뷰 데이터 로드
predicted_reviews_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/skin_toner.csv')
predicted_reviews_df.rename(columns={
    'label': 'sentiment',
    'review': 'txt',

}, inplace=True)

In [ ]:
predicted_reviews_df['review'] = predicted_reviews_df['txt'].str.replace('\n', ' ').replace('\r', ' ')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)  # 감성 라벨이 긍정/부정 두개이므로 num_labels=2으로 설정

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class ReviewDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len

        # 토큰화한 길이를 계산하고, max_len보다 큰 데이터를 필터링합니다.
        self.data = data[data['txt'].apply(lambda x: len(self.tokenizer.tokenize(x)) < self.max_len)]

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['txt']
        label = row['sentiment']

        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'ids': inputs['input_ids'].flatten(),
            'mask': inputs['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return len(self.data)

In [ ]:
# 로더 준비
predicted_dataset = ReviewDataset(predicted_reviews_df, tokenizer, max_len=512)
predicted_loader = DataLoader(predicted_dataset, batch_size=16, shuffle=False)

Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/Colab Notebooks/L25_drop02_10epoch_sche0_4_best_model.pth'))
model.eval()
preds = []
for batch in predicted_loader:
    input_ids = batch['ids']
    attention_mask = batch['mask']
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    preds.extend(torch.argmax(outputs.logits, axis=-1).tolist())


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)  # 감성 라벨이 긍정/부정 두개이므로 num_labels=2으로 설정

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
max_len = 512
filtered_data =predicted_reviews_df[predicted_reviews_df['txt'].apply(lambda x: len(tokenizer.tokenize(x)) < max_len)]
results_df = pd.DataFrame({'name': filtered_data['name'], 'review': filtered_data['txt'], 'sentiment': preds})

In [ ]:
# 긍정적인 리뷰와 부정적인 리뷰 선택
positive_reviews = results_df[results_df['sentiment'] == 1]
negative_reviews = results_df[results_df['sentiment'] == 0]


In [ ]:
# 각 상품명에 따라 긍정적인 리뷰와 부정적인 리뷰 그룹화
positive_grouped = positive_reviews.groupby('name')
negative_grouped = negative_reviews.groupby('name')

In [ ]:
# 각 그룹을 출력하고, 별도의 파일로 저장
for name, group in positive_grouped:
    print(f"Product name: {name}")
    print("Positive reviews:")
    print(group)
    #group.to_csv(os.path.join( f'{name}_positive_reviews.csv'), index=False)

for name, group in negative_grouped:
    print(f"Product name: {name}")
    print("Negative reviews:")
    print(group)
    #group.to_csv(os.path.join( f'{name}_negative_reviews.csv'), index=False)


Product name: [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(350ML+350ML리필)
Positive reviews:
                                                 name  \
4   [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
9   [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
17  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
20  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
40  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
50  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
51  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
52  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
53  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
54  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
55  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
56  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
57  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
58  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...   
59  [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(3...

In [ ]:
# TF-IDF Vectorizer 초기화
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

In [ ]:
def extract_top_keywords_with_tfidf(grouped_reviews, n=10):
    # 각 그룹의 리뷰를 하나의 문서로 결합
    documents = [' '.join(group['review'].tolist()) for _, group in grouped_reviews]
    names = [name for name, _ in grouped_reviews]

    # TF-IDF 값 계산
    tfidf_matrix = vectorizer.fit_transform(documents)

    top_keywords = {}
    for i, name in enumerate(names):
        row = tfidf_matrix.getrow(i).toarray()[0]
        top_n_indices = row.argsort()[-n:][::-1]
        top_n_keywords = [vectorizer.get_feature_names_out()[idx] for idx in top_n_indices]
        top_keywords[name] = top_n_keywords

    return top_keywords


In [ ]:
# 각 상품별 긍정 및 부정 리뷰에서 중요한 키워드 추출
positive_keywords = extract_top_keywords_with_tfidf(positive_grouped, n=5)
negative_keywords = extract_top_keywords_with_tfidf(negative_grouped, n=3)

In [ ]:
print("Top Keywords for Positive Reviews:")
for name, keywords in positive_keywords.items():
    print(f"Product {name}: {', '.join(keywords)}")

print("\nTop Keywords for Negative Reviews:")
for name, keywords in negative_keywords.items():
    print(f"Product {name}: {', '.join(keywords)}")

Top Keywords for Positive Reviews:
Product [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(350ML+350ML리필): 같아요, 어성초, 아누아, 토너, 있는
Product [8월올영픽/대용량기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60매 증정): 다이브인, 토리든, 토너, 독도토너, 좋아요
Product [단독기획] 넘버즈인 3번 결광가득 에센스 토너 200ml (화장솜 증정): 넘버즈인, 에센스, 3번, 저는, 너무
Product [쿨링진정] 넘버즈인 1번 진정 맑게담은 청초토너 300ml 기획 (+1번 패드 10매 증정): 무색, 토너여서, 이벤트로, 쟁입니다, 닦아줄
Product [한정기획/대용량] 라운드랩 1025 독도 토너 500ml+200ml 기획(+소나무 클렌저 10ml 증정): 독도, 좋아요, 토너, 같아요, 많이
Product [한정기획] 라운드랩 1025 독도 토너 대용량 리필기획 (300ml+300ml 리필+약콩판테놀크림1.5ml): 독도, 좋아요, 토너, 같아요, 많이
Product 아누아 어성초 77 수딩 토너 350ml 기획(+토너40ml+패드2매+선크림10ml 증정): 아누아, 토너, 어성초, 같아요, 너무
Product 아비브 어성초 카밍 토너 스킨부스터 250ml 기획(+250ml 리필증정): 아비브, 어성초, 토너를, 토너, 좋아요

Top Keywords for Negative Reviews:
Product [8월 올영픽/리필기획] 아누아 어성초 77 수딩 토너 350ML 리필 기획세트(350ML+350ML리필): 어성초, 피부가, 토너
Product [8월올영픽/대용량기획] 토리든 다이브인 저분자 히알루론산 토너 500ml 기획 (+화장솜 60매 증정): 토리든, 너무, 배송
Product [단독기획] 넘버즈인 3번 결광가득 에센스 토너 200ml (화장솜 증정): 발효, 피부, 너무
Product [쿨링진정] 넘버즈인 1번